In [2]:
import audiofile

In [3]:
import os

In [4]:
import numpy as np

In [5]:
import matplotlib.pyplot as plt

In [6]:
import matplotlib.lines as plt_lines

In [7]:
from scipy.fft import rfft, irfft

In [8]:
from scipy.signal import hilbert, savgol_filter, spectrogram

In [9]:
from sklearn.preprocessing import minmax_scale

In [10]:
AUDIO_FILE = '/Users/danielazulay/Developer/subliminalindustries/recordings/audio/HearBoost Phone/14-6-2023/2023-06-14_1'
signal, fs = audiofile.read(f'{AUDIO_FILE}.wav')
if signal.ndim == 2 and signal.shape[0] == 2:
    signal = signal[0]
signal = minmax_scale(signal, (-1., 1.))
signal.shape

(353933349,)

In [11]:
def entropy(data):
    if data.size < 2:
        return 0.
        
    data = data[data.nonzero()]
    P = data / np.sum(data)
    H = -np.sum(P * np.log2(P))
    return H / np.log2(24)

In [13]:
def fft_ssss_enhancer(data, fs, n=512, ts=32):
    print(f'- data contains {data.shape[0]} samples')

    F = []
    for s in range(0, data.size-1, n):
        d = min(data.size-1, n)
        F.append(rfft(data[s:s+d], n=n)[1:])
    F = np.array(F).T
    (f, t) = F.shape
    
    # print(f'- calculating spectrogram (fs={fs}, N={n})..')
    # f, t, Pxx = spectrogram(x=data, fs=fs, nfft=n, mode='psd')
    
    H = np.ndarray(shape=F.shape) # horizontal spectral entropy
    V = np.ndarray(shape=F.shape) # vertical spectral entropy

    # TODO: use complex spectrogram and multiply real part with V and H then add imaginary part from original to preserve phase information.
    print(f"- calculating horizontal spectral entropy..")
    for freq in range(0, f-1):
        e = np.ndarray(shape=(t,))
        e.fill(entropy(F[freq, :]).real)
        H[freq, :] = e
        
    # print(f'- calculating vertical spectral entropy..')
    # for step in range(0, t-1):
    #     e = np.ndarray(shape=(f,))
    #     e.fill(entropy(F[:, step]).real)
    #     V[:, step] = e

    print(f'- calculating inter-band entropy difference measure..')
    smallfloat = np.finfo(np.float64).tiny
    #E = minmax_scale(H * V, (0., 1.))
    E = minmax_scale(H, (0., 1.))
    # E = minmax_scale(V, (0., 1.))

    bands = np.copy(E)
    for freq in range(0, f-1):
        for cfreq in range(0, f-1):
            if cfreq == freq:
                bands[cfreq, :] = E[cfreq, :]
                continue
                
            bands[cfreq, :] = np.abs(np.square(np.sqrt(np.nan_to_num(E[cfreq, :], nan=smallfloat, posinf=1.-smallfloat, neginf=0.+smallfloat))
                                               - np.sqrt(np.nan_to_num(E[freq, :], nan=smallfloat, posinf=1.-smallfloat, neginf=0.+smallfloat))))
    E = minmax_scale(bands, (0., 1.))

    # print(f'- filtering entropy..')
    # for freq in range(0, f-1):
    #     E[freq, :] = 1-savgol_filter(E[freq, :], t, 2)

    # for step in range(0, t-1):
    #     E[:, step] = 1-savgol_filter(E[:, step], f, 2)
    
    print('- multiplying in frequency domain..')
    for freq in range(0, f-1):
        F[freq] = F[freq] * E[freq]
        
    print('- applying inverse fft..')
    outE = []
    out = []
    for step in range(0, t-1):
        outE.append(irfft(E[:, step], n=n))
        out.append(irfft(F[:, step], n=n))
        
    out = np.block(out)
    outE = np.block(outE)
    out = minmax_scale(np.interp(np.linspace(0, out.size-1, num=data.shape[0]), np.arange(out.size), out), (-1., 1.))
    outE = minmax_scale(np.interp(np.linspace(0, outE.size-1, num=data.shape[0]), np.arange(outE.size), outE), (-1., 1.))
    print(out.shape, outE.shape)

    return (out, outE)

In [14]:
out, e = fft_ssss_enhancer(signal, fs, n=1024, ts=32)

- data contains 353933349 samples
- calculating horizontal spectral entropy..
- calculating inter-band entropy difference measure..
- multiplying in frequency domain..
- applying inverse fft..
(353933349,) (353933349,)


In [ ]:
f = plt.figure()
f.subplots_adjust(left=0.1, right=0.9, top=2.9, hspace=0.6)
f.set_figheight(6)
f.set_figwidth(12)
plt.suptitle(f'Analysis of "{AUDIO_FILE}.wav"', y=3.)

T = np.linspace(0, signal.size / fs, signal.size, endpoint=False)
ax_sig = f.add_subplot(511)
ax_sig.margins(x=0)
ax_sig.set_ylim(bottom=-1., top = 1.)
ax_sig.set_title('Input Signal')
ax_sig.set_ylabel('Amplitude')
ax_sig.set_xlabel('Time (s)')
ax_sig.plot(T, signal, color='#333333', linewidth=.1)

ax_psd_spec = f.add_subplot(512)
ax_psd_spec.margins(x=0)
ax_psd_spec.set_title('Input Signal PSD Spectrum')
ax_psd_spec.set_ylabel('Frequency (Hz)')
ax_psd_spec.set_xlabel('Time (s)')
ax_psd_spec.specgram(signal, NFFT=1024, Fs=fs, window=np.bartlett(1024), noverlap=900, mode='psd', cmap='nipy_spectral')

T = np.linspace(0, e.size / fs, e.size, endpoint=False)
ax_ent = f.add_subplot(513)
ax_ent.margins(x=0)
ax_ent.set_ylim(bottom=0., top = 1.)
ax_ent.set_title('Spectral Entropy')
ax_ent.set_ylabel('Entropy')
ax_ent.set_xlabel('Time (s)')
ax_ent.plot(T, e, color='#333333', linewidth=.1)

ax_se = f.add_subplot(514)
ax_se.margins(x=0)
ax_se.set_ylim(bottom=-1., top = 1.)
ax_se.set_title('Output Signal')
ax_se.set_ylabel('Amplitude')
ax_se.set_xlabel('Time (s)')
ax_se.plot(T, out, color='magenta', linewidth=.1)

ax_psd_spec = f.add_subplot(515)
ax_psd_spec.margins(x=0)
ax_psd_spec.set_title('Output Signal PSD Spectrum')
ax_psd_spec.set_ylabel('Frequency (Hz)')
ax_psd_spec.set_xlabel('Time (s)')
ax_psd_spec.specgram(out, NFFT=1024, Fs=fs, window=np.bartlett(1024), noverlap=900, mode='psd', cmap='nipy_spectral')

print()

In [1]:
if os.path.isfile(f'{AUDIO_FILE}-entropy.wav'):
    os.remove(f'{AUDIO_FILE}-entropy.wav')
audiofile.write(f'{AUDIO_FILE}-entropy.wav', out, fs, bit_depth=24, normalize=True)

NameError: name 'os' is not defined